In [1]:
import pandas as pd
import re

In [2]:
loan_lenders = pd.read_csv("additional-kiva-snapshot\loans_lenders.csv")
country_stats = pd.read_csv("additional-kiva-snapshot\country_stats.csv")
lenders = pd.read_csv("additional-kiva-snapshot\lenders.csv")
loan = pd.read_csv("additional-kiva-snapshot\loans.csv", parse_dates = ['planned_expiration_time', 'disburse_time'])

### 1 Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

Viene inserito un nuovo attributo denominato "n_lenders" in modo da sapere quanti creditori hanno preso parte a quel prestito (tornerà utile nel punto 8 dato che si presuppone che ogni creditore contribuisca equamente al prestito).

La normalizzazione effettuata si riferisce all'atomicità dei singoli attributi.

In [3]:
loan_lenders['n_lenders'] = loan_lenders.lenders.apply(lambda x: len(str(x).split(',')))

In [4]:
from itertools import chain

In [5]:
temp = loan_lenders['lenders'].str.split(',')
loan_lenders_norm = pd.DataFrame({
    'loan_id' : loan_lenders['loan_id'].values.repeat(temp.str.len()),
    'lenders' : list(chain.from_iterable(temp.tolist())),
    'n_lenders' : loan_lenders['n_lenders'].values.repeat(temp.str.len())
})

In [6]:
loan_lenders_norm.head(15)

,loan_id,lenders,n_lenders
0,483693,muc888,40
1,483693,sam4326,40
2,483693,camaran3922,40
3,483693,lachheb1865,40
4,483693,rebecca3499,40
5,483693,karlheinz4543,40
6,483693,jerrydb,40
7,483693,paula8951,40
8,483693,gmct,40
9,483693,amra9383,40


In [7]:
loan_lenders_norm.tail(15)

,loan_id,lenders,n_lenders
28293916,1206220,marilyn1116,44
28293917,1206220,sheila5822,44
28293918,1206220,sebastien24117895,44
28293919,1206220,martin19411599,44
28293920,1206220,florence9711,44
28293921,1206425,rich6705,8
28293922,1206425,sergiiy9766,8
28293923,1206425,angela7509,8
28293924,1206425,barbara5610,8
28293925,1206425,david9667,8


### 2 For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.

In [8]:
loan.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

In [9]:
loan['duration'] = loan['planned_expiration_time'] - loan['disburse_time']
loan['duration'] = loan['duration'].dt.days

### 3 Find the lenders that have funded at least twice.

In [10]:
loan_funded = loan[loan["status"]=="funded"]
loan_funded_merged = pd.merge(loan_lenders_norm, loan_funded["loan_id"], left_on="loan_id", right_on="loan_id")
times_founded = loan_funded_merged["lenders"].value_counts()

In [11]:
times_founded[times_founded >= 2]

 gooddogg1           142349
 trolltech4460       139191
 gmct                122156
 nms                  93283
 themissionbeltco     70151
 rjhoward1986         46965
 joinFITE             37892
 barbara5610          36485
 wakibi               31998
 amirali5409          28181
 highgrovechurch      26709
 shirley1905          26448
 will8174             25901
gooddogg1             25163
 lidia5529            22931
 laurence5353         22232
 aaron83574977        22206
 hans8594             21934
 szucheng2565         21483
 don9212              20891
 maria44596761        20242
 andrewhoffman        19935
 howard7012           19818
 craig97299604        19371
 rene3075             19154
 am8748               18734
 bob4739              18685
 judith5696           18118
 thierry8170          18117
 howard4162           18002
                      ...  
 solveig1594              2
 dean4149                 2
 deborah28978988          2
superheman                2
 abhilasha8494      

### 4 For each country, compute how many loans have involved that country as borrowers.

In [12]:
loan.groupby('country_name')['loan_id'].count()

country_name
Afghanistan                               2337
Albania                                   3075
Armenia                                  13952
Azerbaijan                               10172
Belize                                     218
Benin                                     5946
Bhutan                                       2
Bolivia                                  25250
Bosnia and Herzegovina                     608
Botswana                                     1
Brazil                                     482
Bulgaria                                   296
Burkina Faso                              3489
Burundi                                   1727
Cambodia                                 79701
Cameroon                                  5085
Canada                                       1
Chad                                        61
Chile                                      892
China                                      134
Colombia                                 33675


### 5 For each country, compute the overall amount of money borrowed.

In [13]:
money_borrowed_per_country = loan.groupby(['country_name', 'country_code'])['loan_amount'].sum()

In [14]:
display(money_borrowed_per_country)

country_name                          country_code
Afghanistan                           AF               1967950.0
Albania                               AL               4307350.0
Armenia                               AM              22950475.0
Azerbaijan                            AZ              14784625.0
Belize                                BZ                150175.0
Benin                                 BJ               3865825.0
Bhutan                                BT                 20000.0
Bolivia                               BO              44226725.0
Bosnia and Herzegovina                BA                477250.0
Botswana                              BW                  8000.0
Brazil                                BR               1192325.0
Bulgaria                              BG                375300.0
Burkina Faso                          BF               4085200.0
Burundi                               BI               5233450.0
Cambodia                              K

### 6 Like the previous point, but expressed as a percentage of the overall amount lent.

In [15]:
loan.groupby('country_name')['loan_amount'].sum() / loan['loan_amount'].sum() * 100

country_name
Afghanistan                             0.166573
Albania                                 0.364586
Armenia                                 1.942589
Azerbaijan                              1.251410
Belize                                  0.012711
Benin                                   0.327214
Bhutan                                  0.001693
Bolivia                                 3.743468
Bosnia and Herzegovina                  0.040396
Botswana                                0.000677
Brazil                                  0.100922
Bulgaria                                0.031766
Burkina Faso                            0.345782
Burundi                                 0.442973
Cambodia                                4.368706
Cameroon                                0.190514
Canada                                  0.004232
Chad                                    0.001699
Chile                                   0.219726
China                                   0.032209
Colombi

### 7 Like the three previous points, but split for each year (with respect to disburse time).

In [16]:
loan['disburse_time_year'] = loan['disburse_time'].dt.year

In [17]:
loan.groupby(['disburse_time_year', 'country_name'])['loan_id'].count()

disburse_time_year  country_name                    
2005.0              Bulgaria                               3
                    Cambodia                               2
                    Ecuador                               10
                    Gaza                                   8
                    Honduras                              72
                    Kenya                                 40
                    Nicaragua                              6
                    Senegal                                3
                    Tanzania                               5
                    Uganda                                54
2006.0              Azerbaijan                            21
                    Bulgaria                              70
                    Cambodia                              58
                    Ecuador                              253
                    Ghana                                 28
                    Honduras    

In [18]:
loan.groupby(['disburse_time_year', 'country_name'])['loan_amount'].sum()

disburse_time_year  country_name                    
2005.0              Bulgaria                               1850.0
                    Cambodia                               1625.0
                    Ecuador                                5000.0
                    Gaza                                   5000.0
                    Honduras                              33300.0
                    Kenya                                 21350.0
                    Nicaragua                              3300.0
                    Senegal                                3225.0
                    Tanzania                               2300.0
                    Uganda                                25900.0
2006.0              Azerbaijan                            15700.0
                    Bulgaria                              84650.0
                    Cambodia                              41950.0
                    Ecuador                              151175.0
                    Gha

In [19]:
loan.groupby(['disburse_time_year', 'country_name'])['loan_amount'].sum() / loan['loan_amount'].sum() * 100

disburse_time_year  country_name                    
2005.0              Bulgaria                            0.000157
                    Cambodia                            0.000138
                    Ecuador                             0.000423
                    Gaza                                0.000423
                    Honduras                            0.002819
                    Kenya                               0.001807
                    Nicaragua                           0.000279
                    Senegal                             0.000273
                    Tanzania                            0.000195
                    Uganda                              0.002192
2006.0              Azerbaijan                          0.001329
                    Bulgaria                            0.007165
                    Cambodia                            0.003551
                    Ecuador                             0.012796
                    Ghana            

### 8 For each lender, compute the overall amount of money lent.

In [20]:
loan_lenders_merged = pd.merge(loan_lenders_norm, loan[["loan_id", "loan_amount"]], left_on="loan_id", right_on="loan_id")
loan_lenders_merged['amount_per_lenders'] = loan_lenders_merged['loan_amount'] / loan_lenders_merged['n_lenders']

In [21]:
loan_lenders_merged.groupby('lenders')['amount_per_lenders'].sum() 

lenders
 000               1672.618411
 00000             1380.693644
 0002              2472.563566
 00mike00            52.631579
 0101craign0101    2623.565117
 0132575            117.678140
 0154884            136.342116
 0161130             49.779412
 0169713            102.488038
 0185429             57.674632
 0197462             27.631579
 0206338             60.602679
 0219854            105.000000
 0221581             55.844156
 0239059             63.300493
 0245597             33.333333
 0256321             57.086268
 0265562             65.714286
 0279282             30.000000
 0288537             27.272727
 0295920             59.303977
 0307987             59.059233
 0312857             30.625000
 0321212            118.213970
 0326lsw           1794.927610
 0332772             66.666667
 0346439            100.625000
 0353400             26.086957
 0367630             31.250000
 0376099             50.446429
                      ...     
zuzana5790           44.117647


### 9 For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [22]:
lenders.columns

Index(['permanent_name', 'display_name', 'city', 'state', 'country_code',
       'member_since', 'occupation', 'loan_because', 'loan_purchase_num',
       'invited_by', 'num_invited'],
      dtype='object')

In [23]:
import random
import numpy as np

In [24]:
country_count = lenders.country_code.value_counts(normalize=True)

missing = lenders['country_code'].isnull()
lenders.loc[missing,'country_code'] = np.random.choice(country_count.index, size=len(lenders[missing]),p=country_count.values)

In [25]:
loan_lenders_merged2 = pd.merge(loan_lenders_norm, lenders[["permanent_name", "country_code"]], left_on="lenders", right_on="permanent_name")
loan_lenders_merged3 = pd.merge(loan_lenders_merged2, loan[["loan_id", "loan_amount"]], left_on="loan_id", right_on="loan_id")

money_lent_per_country = loan_lenders_merged3.groupby("country_code")["loan_amount"].sum()
money_lent_per_country = pd.DataFrame(money_lent_per_country).reset_index()
money_lent_per_country.columns = ['country_code', 'money_lent']

money_borrowed_per_country = pd.DataFrame(money_borrowed_per_country).reset_index()
money_borrowed_per_country.columns = ['country_name', 'country_code', 'money_borrowed']
lent_borrowed = pd.merge(money_lent_per_country, money_borrowed_per_country[["country_code", "money_borrowed"]], left_on="country_code", right_on="country_code", how="outer")
lent_borrowed.fillna(0, inplace=True)
lent_borrowed["difference"] = lent_borrowed["money_lent"] - lent_borrowed["money_borrowed"] 

In [26]:
display(lent_borrowed)

,country_code,money_lent,money_borrowed,difference
0,AD,2575.0,0.0,2575.0
1,AE,2112250.0,0.0,2112250.0
2,AF,115075.0,1967950.0,-1852875.0
3,AG,1125.0,0.0,1125.0
4,AI,200.0,0.0,200.0
5,AL,37225.0,4307350.0,-4270125.0
6,AM,31050.0,22950475.0,-22919425.0
7,AN,32350.0,0.0,32350.0
8,AO,55975.0,0.0,55975.0
9,AQ,42000.0,0.0,42000.0


### 10 Which country has the highest ratio between the difference computed at the previous point and the population?

In [27]:
country_stats.columns

Index(['country_name', 'country_code', 'country_code3', 'continent', 'region',
       'population', 'population_below_poverty_line', 'hdi', 'life_expectancy',
       'expected_years_of_schooling', 'mean_years_of_schooling', 'gni',
       'kiva_country_name'],
      dtype='object')

In [28]:
lent_borrowed_merged = pd.merge(lent_borrowed[["country_code", "difference"]], country_stats[["country_code", "population"]], left_on="country_code", right_on="country_code")
lent_borrowed_merged["ratio"] = lent_borrowed_merged["difference"] / lent_borrowed_merged["population"]

In [29]:
display(lent_borrowed_merged.sort_values(by="ratio", ascending=False))

,country_code,difference,population,ratio
23,CA,114429250.0,36624199,3.124416
108,NO,14951250.0,5305383,2.818128
68,IS,782500.0,335025,2.335647
7,AU,53833750.0,24450561,2.201739
152,US,664477550.0,324459463,2.047952
107,NL,31167350.0,17035938,1.829506
130,SE,15047800.0,9910701,1.518339
87,LU,845950.0,583455,1.449898
25,CH,11076725.0,8476005,1.306833
46,FI,6101300.0,5523231,1.104661


### 11 Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [30]:
country_stats[country_stats["population_below_poverty_line"].isnull()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
151,Australia,AU,AUS,Oceania,Australia and New Zealand,24450561,NaN,0.938680,82.537,20.43272,13.175100,42822.196270,Australia
152,Bahrain,BH,BHR,Asia,Western Asia,1492584,NaN,0.823884,76.715,14.50000,9.419000,37236.363040,Bahrain
153,Brunei,BN,BRN,Asia,South-eastern Asia,428697,NaN,0.864937,79.019,14.94486,9.020000,72843.466090,Brunei
154,Cuba,CU,CUB,Americas,Caribbean,11484636,NaN,0.774669,79.573,13.93739,11.750781,7455.062477,Cuba
155,Cyprus,CY,CYP,Asia,Western Asia,1179551,NaN,0.855646,80.332,14.25613,11.695230,29458.522690,Cyprus
156,Finland,FI,FIN,Europe,Northern Europe,5523231,NaN,0.894517,81.006,17.04792,11.194000,38868.135520,Finland
157,Iceland,IS,ISL,Europe,Northern Europe,335025,NaN,0.921110,82.724,18.99422,12.190892,37065.214510,Iceland
158,Kuwait,KW,KWT,Asia,Western Asia,4136528,NaN,0.800183,74.549,13.26000,7.265369,76075.207350,Kuwait
159,Luxembourg,LU,LUX,Europe,Western Europe,583455,NaN,0.898465,81.881,13.86373,11.950833,62470.591290,Luxembourg
160,New Zealand,NZ,NZL,Oceania,Australia and New Zealand,4705818,NaN,0.914900,82.026,19.22820,12.503000,32870.078360,New Zealand


Ci sono 22 valori nulli nella variabile "population_below_poverty_line", pertanto si cercherà di rimpiazzarli.
La fonte utilizzata per ricavare la percentuale di popolazione al di sotto della soglia di povertà in questo dataset è presente negli archivi della CIA. Confrontando tali dati con quelli della World Bank i valori osservati erano praticamente identici, quindi, dove possibile, i nostri missing values verranno sostituiti dal valore riportato nelle classifiche della World Bank. Si ottiene quanto segue: 

* Australia 1.2 
* Cyprus 0.2 
* Finland 0.2
* Iceland 0.2
* Luxembourg 0.5
* Norway 0.2
* Solomon Island 56.2
* Central Africa Republic 80.7
* Samoa 14.9
* Palestina 9.4

In [31]:
country_stats.loc[country_stats.country_code == "AU", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "AU", "population_below_poverty_line"].fillna(1.2)
country_stats.loc[country_stats.country_code == "CY", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "CY", "population_below_poverty_line"].fillna(0.2)
country_stats.loc[country_stats.country_code == "FI", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "FI", "population_below_poverty_line"].fillna(0.2)
country_stats.loc[country_stats.country_code == "IS", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "IS", "population_below_poverty_line"].fillna(0.2)
country_stats.loc[country_stats.country_code == "LU", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "LU", "population_below_poverty_line"].fillna(0.5)
country_stats.loc[country_stats.country_code == "NO", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "NO", "population_below_poverty_line"].fillna(0.2)
country_stats.loc[country_stats.country_code == "SB", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "SB", "population_below_poverty_line"].fillna(56.2)
country_stats.loc[country_stats.country_code == "CF", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "CF", "population_below_poverty_line"].fillna(80.7)
country_stats.loc[country_stats.country_code == "WS", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "WS", "population_below_poverty_line"].fillna(14.9)
country_stats.loc[country_stats.country_code == "PS", "population_below_poverty_line"] = country_stats.loc[country_stats.country_code == "PS", "population_below_poverty_line"].fillna(9.4)

Dopo tale operazione tuttavia sono ancora presenti Stati con il valore di nostro interesse che risulta mancante. Esso verrà quindi sostituito con il valore mediano relativo agli altri Stati che appartengono allo stesso continente. La scelta è ricaduta sulla mediana per evitare le eventuali influenze relative a valori molto estremi.

In [32]:
cs_group = country_stats.groupby("continent")["population_below_poverty_line"].median()
cs_group = pd.DataFrame(cs_group).reset_index()
cs_group.columns = ['continent', 'population_below_poverty_line']

In [33]:
cs_group

,continent,population_below_poverty_line
0,Africa,45.1
1,Americas,26.7
2,Asia,19.5
3,Europe,15.0
4,Oceania,31.0


In [34]:
country_stats.loc[country_stats.continent == "Africa", 'population_below_poverty_line'] = country_stats.loc[country_stats.continent == "Africa", 'population_below_poverty_line'].fillna(cs_group.iloc[0,1])
country_stats.loc[country_stats.continent == "Americas", 'population_below_poverty_line'] = country_stats.loc[country_stats.continent == "Americas", 'population_below_poverty_line'].fillna(cs_group.iloc[1,1])
country_stats.loc[country_stats.continent == "Asia", 'population_below_poverty_line'] = country_stats.loc[country_stats.continent == "Asia", 'population_below_poverty_line'].fillna(cs_group.iloc[2,1])
country_stats.loc[country_stats.continent == "Oceania", 'population_below_poverty_line'] = country_stats.loc[country_stats.continent == "Oceania", 'population_below_poverty_line'].fillna(cs_group.iloc[4,1])

In [35]:
country_stats["population_not_below_poverty_line"] = country_stats["population"] - (country_stats["population"] / 100 * country_stats["population_below_poverty_line"])

lent_borrowed_merged2 = pd.merge(lent_borrowed[["country_code", "difference"]], country_stats[["country_code", "population_not_below_poverty_line"]], left_on="country_code", right_on="country_code")
lent_borrowed_merged2["ratio"] = lent_borrowed_merged2["difference"] / lent_borrowed_merged2["population_not_below_poverty_line"]

In [36]:
display(lent_borrowed_merged2.sort_values(by="ratio", ascending=False))

,country_code,difference,population_not_below_poverty_line,ratio
23,CA,114429250.0,3.318152e+07,3.448583
108,NO,14951250.0,5.294772e+06,2.823776
152,US,664477550.0,2.754661e+08,2.412194
68,IS,782500.0,3.343550e+05,2.340327
7,AU,53833750.0,2.415715e+07,2.228481
107,NL,31167350.0,1.553678e+07,2.006037
130,SE,15047800.0,8.424096e+06,1.786281
87,LU,845950.0,5.805377e+05,1.457184
25,CH,11076725.0,7.916589e+06,1.399179
110,NZ,4332850.0,3.247014e+06,1.334410


### 12 For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [37]:
from pandas.tseries import offsets                                     

In [38]:
loan_one_year = loan[loan.disburse_time_year == loan.planned_expiration_time.dt.year]     #record che hanno disburse_time e planned_expiration_time nello stesso anno
loan_more_years = loan[loan.disburse_time_year != loan.planned_expiration_time.dt.year]   #record che hanno disburse_time e planned_expiration_time in anni diversi

In [39]:
total_amount_one_year = loan_one_year.groupby('disburse_time_year')['loan_amount'].sum()

total_amount_one_year = pd.DataFrame(total_amount_one_year).reset_index()
total_amount_one_year.columns = ['year', 'loans_amount']

In [40]:
loan_more_years['expiration_time_year'] = loan_more_years['planned_expiration_time'].dt.year
loan_more_years['n_years'] = loan_more_years['expiration_time_year'] - loan_more_years['disburse_time_year']  #aggiungo variabile per sapere quanti anni trascorrono fra disburse_time e planned_expiration_time

C:\Users\Francesca\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Francesca\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
df = loan_more_years[loan_more_years.n_years >= 0]   #mantengo solo i dati senza errori (ovvero con le date non invertite, per evitare numeri negativi)

In [42]:
df.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model',
       'duration', 'disburse_time_year', 'expiration_time_year', 'n_years'],
      dtype='object')

In [43]:
df = df.loc[df.index.repeat(df['n_years'] + 1)]    #si ripete ogni loan per ogni anno trascorso tra disburse_time e planned_expiration_time
df['year'] = df.groupby(level=0).cumcount() + df.disburse_time_year   #si aggiunge una variabile che farà riferimento all'anno che consideriamo
df['n_years'] = df['n_years'] + 1 
df['end_of_first_year'] = df.disburse_time + offsets.YearEnd(n=0)   #si calcola la data della fine dell'anno relativa a disburse_time 
df['beginning_of_last_year'] = df.planned_expiration_time - offsets.YearBegin()    #si calcola la data relativa all'inizio dell'anno relativa a planned_expiration_time
df['days_first_year'] = (df.end_of_first_year - df.disburse_time).dt.days    #numero di giorni del primo anno
df['days_last_year'] = (df.planned_expiration_time - df.beginning_of_last_year).dt.days    #numero di giorni dell'ultimo anno

In [44]:
#si definisce una funzione per calcolare l'ammontare relativo ad ogni anno tenendo in considerazione che l'ammontare
#va distribuito equamente fra i vari anni coperti

def calculate_amount_per_year(row):
    if row.year==row.disburse_time_year:
        return (row.loan_amount * row.days_first_year) / (row.days_first_year + 365 * (row.n_years - 2) + row.days_last_year)
    elif row.year==row.expiration_time_year:
        return (row.loan_amount * row.days_last_year) / (row.days_first_year + 365 * (row.n_years - 2) + row.days_last_year)
    else:
        return (row.loan_amount * 365) / (row.days_first_year + 365 * (row.n_years - 2) + row.days_last_year)

In [45]:
df['amount_per_year'] = df.apply(calculate_amount_per_year, axis=1)
total_amount_more_years = df.groupby('year')['amount_per_year'].sum()

total_amount_more_years = pd.DataFrame(total_amount_more_years).reset_index()
total_amount_more_years.columns = ['year', 'loans_amount']

In [46]:
total_amount = pd.concat([total_amount_one_year, total_amount_more_years], ignore_index=True)
total_amount.groupby('year')['loans_amount'].sum()

year
2011.0    6.394771e+05
2012.0    1.132873e+08
2013.0    1.241966e+08
2014.0    1.538371e+08
2015.0    1.596145e+08
2016.0    1.589562e+08
2017.0    1.718975e+08
2018.0    5.879006e+06
Name: loans_amount, dtype: float64

### 1B For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state

In [47]:
lenders.columns

Index(['permanent_name', 'display_name', 'city', 'state', 'country_code',
       'member_since', 'occupation', 'loan_because', 'loan_purchase_num',
       'invited_by', 'num_invited'],
      dtype='object')

In [48]:
repayment_group = pd.merge(loan_lenders_merged[["loan_id", "lenders", "amount_per_lenders"]], loan[["loan_id", "repayment_interval"]], left_on="loan_id", right_on="loan_id")
lenders_grouped = repayment_group.groupby(["lenders", "repayment_interval"])["amount_per_lenders"].sum()
lenders2 = lenders_grouped.unstack()
lenders_update = pd.merge(lenders, lenders2, left_on="permanent_name", right_on="lenders", how="left")
lenders_update[["weekly","monthly", "bullet", "irregular"]] = lenders_update[["weekly","monthly", "bullet", "irregular"]].fillna(0)

In [49]:
display(lenders_update)

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,bullet,irregular,monthly,weekly
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0,0.000000,0.000000,0.000000,0.0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0,0.000000,0.000000,0.000000,0.0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0,0.000000,0.000000,0.000000,0.0
3,andrew86079135,Andrew,NaN,NaN,SE,1461301091,NaN,NaN,5.0,Peter Tan,0,0.000000,0.000000,0.000000,0.0
4,nguyen6962,Nguyen,NaN,NaN,CA,1461301154,NaN,NaN,NaN,NaN,0,0.000000,0.000000,0.000000,0.0
5,sirinapa6764,Sirinapa,NaN,NaN,US,1461301496,NaN,NaN,1.0,NaN,0,0.000000,0.000000,0.000000,0.0
6,rene7585,Rene,NaN,NaN,CO,1461301636,NaN,NaN,2.0,NaN,0,0.000000,0.000000,0.000000,0.0
7,harald2826,Harald,NaN,NaN,US,1461301670,NaN,NaN,2.0,NaN,0,0.000000,0.000000,0.000000,0.0
8,mehdi2903,Mehdi,NaN,NaN,US,1461301756,NaN,NaN,NaN,NaN,0,0.000000,0.000000,0.000000,0.0
9,youchan8125,Youchan,NaN,NaN,US,1461301941,NaN,NaN,1.0,NaN,0,0.000000,0.000000,0.000000,0.0


### 2B What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?

In [50]:
lenders.columns

Index(['permanent_name', 'display_name', 'city', 'state', 'country_code',
       'member_since', 'occupation', 'loan_because', 'loan_purchase_num',
       'invited_by', 'num_invited'],
      dtype='object')

In [51]:
lenders2 = lenders[pd.notnull(lenders["occupation"])]
loan_lenders2_merged = pd.merge(loan_lenders_merged[["loan_id", "amount_per_lenders", "lenders"]], lenders2[["permanent_name", "occupation"]], left_on="lenders", right_on="permanent_name")

Ci sono occupazioni scritte in modo differente, ad esempio "Teacher" e "teacher" oppure "Artist" e "artist".

In [52]:
df = loan_lenders2_merged.assign(occupation=loan_lenders2_merged['occupation'].str.capitalize()).groupby("occupation", as_index=False).mean()

In [53]:
df.loc[df['amount_per_lenders'].idxmax()][["occupation", "amount_per_lenders"]]

occupation            Publisher, skirt! magazine
amount_per_lenders                          5075
Name: 19014, dtype: object

### 3B Cluster the loans according to the year-month of disburse time.

In [54]:
loan['disburse_time_month'] = loan['disburse_time'].dt.month

In [55]:
cluster = loan.groupby(['disburse_time_year', 'disburse_time_month']).apply(lambda x: x["loan_id"].tolist()).to_dict()

Per suddividere i vari _loans_ è stato creato un dizionario che avesse come chiave l'anno e il mese di _disburse_time_. A causa della grande quantità di dati, verranno riportati gli output relativi a due chiavi come esempio. 

In [60]:
print(cluster[(2006.0, 6.0)])

[457, 439, 436, 470, 438, 372, 477, 393, 448, 447, 431, 476, 461, 450, 396, 441, 467, 434, 462, 463, 442, 397, 468, 444, 375, 466, 458, 418, 395, 465, 460, 394, 475, 464, 433, 491, 451, 365, 489, 472, 440, 391, 427, 452, 454, 443, 459, 428, 445, 449, 469, 435, 478, 417, 366, 455, 432, 408, 446, 490, 421, 473, 413, 425, 437, 495]


In [61]:
print(cluster[(2006.0, 7.0)])

[480, 514, 532, 497, 527, 390, 494, 548, 559, 542, 364, 546, 523, 499, 544, 389, 474, 508, 374, 516, 522, 482, 424, 498, 551, 535, 481, 545, 487, 493, 471, 501, 505, 531, 456, 479, 500, 536, 503, 539, 509, 507, 515, 547, 538, 525, 518, 513, 560, 492, 521, 530, 543, 369, 529, 483, 519, 486, 485, 512, 504, 557, 537, 506, 549, 496, 511, 534, 502, 488, 520, 533]


### 4B For each country, compute its overall GDP, by multiplying the per capita GDP with its population.

In [62]:
country_stats.columns

Index(['country_name', 'country_code', 'country_code3', 'continent', 'region',
       'population', 'population_below_poverty_line', 'hdi', 'life_expectancy',
       'expected_years_of_schooling', 'mean_years_of_schooling', 'gni',
       'kiva_country_name', 'population_not_below_poverty_line'],
      dtype='object')

In [63]:
country_stats['overall_gni'] = country_stats['gni'] * country_stats['population']

In [64]:
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,population_not_below_poverty_line,overall_gni
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,1.045900e+09,7.584413e+12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,5.726589e+07,1.038975e+12
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,6.948984e+07,2.116096e+12
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,1.388962e+08,9.912214e+11
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,1.127988e+08,5.502424e+11


In [65]:
country_stats.tail()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,population_not_below_poverty_line,overall_gni
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,45.1,NaN,NaN,NaN,NaN,NaN,Somalia,8093645.127,NaN
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,80.7,0.35244,51.458,7.09898,4.23,587.473961,Central African Republic,899202.440,2.737088e+09
171,Samoa,WS,WSM,Oceania,Polynesia,196440,14.9,0.70200,NaN,NaN,NaN,NaN,Samoa,167170.440,NaN
172,Palestine,PS,PS,Asia,Western Asia,4920724,9.4,0.67700,NaN,NaN,NaN,NaN,Palestine,4458175.944,NaN
173,Kosovo,XK,NaN,Europe,Southern Europe,1895250,30.0,0.78600,NaN,NaN,NaN,NaN,Kosovo,1326675.000,NaN
